<a href="https://colab.research.google.com/github/yesong98/Exploration/blob/main/Visual_Flix_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
! git clone https://github.com/SBShimm/Datathon.git

fatal: destination path 'Datathon' already exists and is not an empty directory.


# 1. 데이터 가져오기
#### 1) 데이터 종류

In [39]:
file_dir = '/content/drive/MyDrive/Visual_Flix/Netflix_IMDB.csv'

data = pd.read_csv(file_dir)

In [40]:
data.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",7.4
1,1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",6.3
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN
3,3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN
4,4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN


# 2. Data 전처리

#### 1)Data 추출

In [41]:
data['date_added'] = pd.to_datetime(data['date_added'])

In [42]:
# datetime 타입에서 년, 월, 일 추출
data['year'] = data.date_added.apply(lambda x : x.year)
data['month'] = data.date_added.apply(lambda x : x.month)
data['day'] = data.date_added.apply(lambda x : x.day)

#년월일 int로 변경
data['year'] = pd.to_datetime(data['year']).astype(np.int64)
data['month'] = pd.to_datetime(data['month']).astype(np.int64)
data['day'] = pd.to_datetime(data['day']).astype(np.int64)

data.head(3)

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB,year,month,day
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",7.4,2021,9,25
1,1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",6.3,2021,9,24
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,2021,9,24


#### 2) Type 나누기, 장르세분화

In [50]:
def genere(what):
    if what == 'all':
        genere = data.listed_in.value_counts()
    else:
        genere = data[data['type'] == what].listed_in.value_counts()
    gener = {}
    
    for idx, val in genere.items():
        l = idx.split(',')
        for i in l:
            i = i.strip()
            if i in gener.keys():
                d = {}
                d[i] = val + gener[i]
                gener.update(d)
            else:
                d = {i:val}
                gener.update(d)
    theme, count = [],[]
    
    for idx, val in gener.items():
        theme.append(idx)
        count.append(val)
    
    list_temp = (pd.DataFrame({'genere':theme, 'count': count})
        .sort_values('count', ascending = True))
    list_temp.reset_index(drop = True, inplace = True)
    
    return list_temp

In [51]:
tv_genere = genere('TV Show')
tv_genere

,genere,count
0,TV Shows,16
1,Classic & Cult TV,28
2,Stand-Up Comedy & Talk Shows,56
3,TV Thrillers,59
4,Teen TV Shows,69
5,TV Horror,76
6,TV Sci-Fi & Fantasy,85
7,Science & Nature TV,92
8,TV Mysteries,101
9,Korean TV Shows,155


In [52]:
mov_genere = genere('Movie')
mov_genere

,genere,count
0,Movies,58
1,Faith & Spirituality,66
2,Cult Movies,71
3,Anime Features,71
4,LGBTQ Movies,103
5,Classic Movies,116
6,Sports Movies,220
7,Sci-Fi & Fantasy,245
8,Stand-Up Comedy,343
9,Horror Movies,358


#### 3)Country data 전처리

In [53]:
data['first_country'] = data.country.str.extract(r'([a-zA-Z\s]+)')

In [54]:
data.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB,year,month,day,first_country
6010,6010,s5941,TV Show,Breaking Bad,NaN,"Bryan Cranston, Aaron Paul, Anna Gunn, Dean No...",United States,2013-08-02,2013,TV-MA,5 Seasons,"Crime TV Shows, TV Dramas, TV Thrillers",A high school chemistry teacher dying of cance...,9.4,2013,8,2,United States
2558,2558,s2520,TV Show,Avatar: The Last Airbender,NaN,"Zach Tyler, Mae Whitman, Jack De Sena, Dee Bra...",United States,2020-05-15,2007,TV-Y7,3 Seasons,"Classic & Cult TV, Kids' TV, TV Action & Adven...",Siblings Katara and Sokka wake young Aang from...,9.3,2020,5,15,United States
3119,3119,s3072,TV Show,Reply 1988,NaN,"Lee Hye-ri, Park Bo-gum, Ryu Jun-yeol, Ko Kyou...",South Korea,2020-01-01,2015,TV-14,1 Season,"International TV Shows, Korean TV Shows, Roman...",Take a nostalgic trip back to the late 1980s t...,9.2,2020,1,1,South Korea
688,688,s681,TV Show,The Last Dance,NaN,"Michael Jordan, Scottie Pippen, Dennis Rodman",United States,2021-06-19,2020,TV-MA,1 Season,Docuseries,This docuseries gives a definitive account of ...,9.2,2021,6,19,United States
3995,3995,s3939,TV Show,Our Planet,NaN,David Attenborough,"United States, United Kingdom",2019-04-05,2019,TV-PG,1 Season,"Docuseries, Science & Nature TV",Experience our planet's natural beauty and exa...,9.2,2019,4,5,United States


#### 4) 별점 data (내림차순)

In [55]:
df_rating = data.sort_values(by = 'IMDB',ascending=False)
df_rating.head(20)

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB,year,month,day,first_country
6010,6010,s5941,TV Show,Breaking Bad,NaN,"Bryan Cranston, Aaron Paul, Anna Gunn, Dean No...",United States,2013-08-02,2013,TV-MA,5 Seasons,"Crime TV Shows, TV Dramas, TV Thrillers",A high school chemistry teacher dying of cance...,9.4,2013,8,2,United States
2558,2558,s2520,TV Show,Avatar: The Last Airbender,NaN,"Zach Tyler, Mae Whitman, Jack De Sena, Dee Bra...",United States,2020-05-15,2007,TV-Y7,3 Seasons,"Classic & Cult TV, Kids' TV, TV Action & Adven...",Siblings Katara and Sokka wake young Aang from...,9.3,2020,5,15,United States
3119,3119,s3072,TV Show,Reply 1988,NaN,"Lee Hye-ri, Park Bo-gum, Ryu Jun-yeol, Ko Kyou...",South Korea,2020-01-01,2015,TV-14,1 Season,"International TV Shows, Korean TV Shows, Roman...",Take a nostalgic trip back to the late 1980s t...,9.2,2020,1,1,South Korea
688,688,s681,TV Show,The Last Dance,NaN,"Michael Jordan, Scottie Pippen, Dennis Rodman",United States,2021-06-19,2020,TV-MA,1 Season,Docuseries,This docuseries gives a definitive account of ...,9.2,2021,6,19,United States
3995,3995,s3939,TV Show,Our Planet,NaN,David Attenborough,"United States, United Kingdom",2019-04-05,2019,TV-PG,1 Season,"Docuseries, Science & Nature TV",Experience our planet's natural beauty and exa...,9.2,2019,4,5,United States
1915,1915,s1887,Movie,David Attenborough: A Life on Our Planet,"Alastair Fothergill, Jonnie Hughes, Keith Scholey",David Attenborough,"United States, United Kingdom",2020-10-04,2020,PG,84 min,Documentaries,"A broadcaster recounts his life, and the evolu...",9.1,2020,10,4,United States
3431,3431,s3382,TV Show,The Untamed,NaN,"Sean Xiao, Wang Yibo, Zoey Meng, Xuan Lu, Yu B...",China,2019-10-25,2019,TV-14,1 Season,"International TV Shows, TV Action & Adventure,...","In a magical world of inter-clan rivalry, two ...",9.1,2019,10,25,China
7365,7365,s7296,TV Show,Leyla and Mecnun,Onur Ünlü,"Ali Atay, Melis Birkan, Serkan Keskin, Ahmet M...",Turkey,2017-04-15,2014,TV-MA,3 Seasons,"International TV Shows, Romantic TV Shows, TV ...",Destiny brings Mecnun and Leyla together as ne...,9.1,2017,4,15,Turkey
836,836,s827,Movie,Bo Burnham: Inside,Bo Burnham,Bo Burnham,United States,2021-05-30,2021,TV-MA,88 min,Stand-Up Comedy,A new comedy special shot and performed by Bo ...,9.0,2021,5,30,United States
2446,2446,s2409,TV Show,My Mister,NaN,"Lee Sun-kyun, Lee Ji-eun (IU), Go Du-sim, Park...",South Korea,2020-06-10,2018,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Korean...","In a world that is less than kind, a young wom...",9.0,2020,6,10,South Korea
